# Constructed Graph

In [1]:
from tqdm import tqdm
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
import pandas as pd 
from itertools import islice
# Create a graph from text file 
# user_id, user_id


In [2]:
df = pd.read_csv('../data/ub_sample_data.csv')
# random_ids = df.user_id.sample(100).unique()
# df = df[df.user_id.isin(random_ids)]

In [3]:
G = nx.Graph()

In [4]:
usr_ids = df.user_id.unique()
business_dict = {}
for u in tqdm(usr_ids):
    business_u = set(df[df.user_id == u].business_id.unique())
    business_dict[u] = business_u

100%|██████████| 3374/3374 [00:18<00:00, 178.48it/s]


In [5]:
usr_ids = df.user_id.unique()
edge_threshold = 7
for u in tqdm(usr_ids):
    for v in tqdm(usr_ids, leave=False):
        if u != v:
            business_u = business_dict[u]
            business_v = business_dict[v]
            if len(business_u.intersection(business_v)) >= edge_threshold:
                G.add_edge(u, v)
        else:
            # if u == v, business cannot be reviewed by the same user
            # G.add_node(u)
            pass

100%|██████████| 3374/3374 [00:50<00:00, 67.06it/s]


In [6]:
len(G.nodes), len(G.edges)

(222, 498)

# Caculate betweeness centrality of original graph

In [7]:
result = nx.edge_betweenness_centrality(G)
result = sorted(result.items(), key=lambda x: x[1], reverse=True)
def sort_by_lexicographical_order(edge):
    return tuple(sorted(edge))
result = [(sort_by_lexicographical_order(edge[0]), edge[1]) for edge in result]
result[:5]

[(('cyuDrrG5eEK-TZI867MUPA', 'l-1cva9rA8_ugLrtSdKAqA'), 0.17259793730381964),
 (('1st2ltGKJ00ZcRsev-Ieew', 'DKolrsBSwMTpTJL22dqJRQ'), 0.1625081448610863),
 (('1st2ltGKJ00ZcRsev-Ieew', 'HLY9oDcVBH9D25lU4X_V5Q'), 0.15722962781786312),
 (('1st2ltGKJ00ZcRsev-Ieew', 'Hv_q_ZnSIoZwdcoH0CyV2Q'), 0.15180791651379885),
 (('1st2ltGKJ00ZcRsev-Ieew', 'JM0GL6Dx4EuZ1mprLk5Gyg'), 0.14958675546910816)]

In [8]:
with open('edge_betweenness_centrality.txt', 'w') as f:
    for item in result:
        f.write(f'({item[0][0]},{item[0][1]}),{item[1]}')
        f.write('\n')
    f.close()
!head edge_betweenness_centrality.txt

(cyuDrrG5eEK-TZI867MUPA,l-1cva9rA8_ugLrtSdKAqA),0.17259793730381964
(1st2ltGKJ00ZcRsev-Ieew,DKolrsBSwMTpTJL22dqJRQ),0.1625081448610863
(1st2ltGKJ00ZcRsev-Ieew,HLY9oDcVBH9D25lU4X_V5Q),0.15722962781786312
(1st2ltGKJ00ZcRsev-Ieew,Hv_q_ZnSIoZwdcoH0CyV2Q),0.15180791651379885
(1st2ltGKJ00ZcRsev-Ieew,JM0GL6Dx4EuZ1mprLk5Gyg),0.14958675546910816
(HLY9oDcVBH9D25lU4X_V5Q,l-1cva9rA8_ugLrtSdKAqA),0.1359504300680771
(Hv_q_ZnSIoZwdcoH0CyV2Q,l-1cva9rA8_ugLrtSdKAqA),0.13407525172231055
(0FVcoJko1kfZCrJRfssfIA,DKolrsBSwMTpTJL22dqJRQ),0.09035984963113673
(a48HhwcmjFLApZhiax41IA,o-t-i7nbT5N_cmkCXs5oDQ),0.08026578614813909
(A-U-K9z9oraMH7eBZW1dOA,l-1cva9rA8_ugLrtSdKAqA),0.07737148913619502


# Community Detection

In [9]:
import time 

In [10]:
comp = girvan_newman(G)
c = next(comp)
nx.algorithms.community.modularity(G, c)   

0.5247012144965406

In [11]:
G_clone = G.copy()
comp = girvan_newman(G_clone)
iter = 0
max_modularity = -1 
max_modularity_communities = None
while True:
    iter += 1 
    start = time.time()
    try:
        communities = next(comp)
    except:
        print('no more communities')
        break
    modularity = nx.algorithms.community.modularity(G_clone, communities)   
    if modularity > max_modularity:
        max_modularity = modularity
        max_modularity_communities = communities
    print(f'iter: {iter}, modularity: {modularity}, time: {time.time() - start}')

iter: 1, modularity: 0.5247012144965406, time: 0.9725151062011719
iter: 2, modularity: 0.6249032273673009, time: 0.8436574935913086
iter: 3, modularity: 0.6490742084805088, time: 0.07246017456054688
iter: 4, modularity: 0.6527394719440013, time: 0.2017812728881836
iter: 5, modularity: 0.6823277043918646, time: 0.16826438903808594
iter: 6, modularity: 0.6872550442734795, time: 0.18161344528198242
iter: 7, modularity: 0.6782713182045451, time: 0.7283084392547607
iter: 8, modularity: 0.6806079740649345, time: 0.15972065925598145
iter: 9, modularity: 0.6814063482847051, time: 0.34764647483825684
iter: 10, modularity: 0.6787451008854696, time: 0.1929771900177002
iter: 11, modularity: 0.6624711698198417, time: 1.2160530090332031
iter: 12, modularity: 0.6536386509895002, time: 0.1643531322479248
iter: 13, modularity: 0.6521931097885519, time: 0.12196612358093262
iter: 14, modularity: 0.6508463573168175, time: 0.026311874389648438
iter: 15, modularity: 0.6493988000193546, time: 0.0197882652282

In [12]:
max_modularity

0.6872550442734795

In [14]:
result = sorted(max_modularity_communities, key=lambda x: len(x), reverse=False)
result[0]

{'Si3aMsOVGSVlsc54iuiPwA', 'd5WLqmTMvmL7-RmUDVKqqQ'}

In [15]:
len(result)

19

In [68]:
with open('girvan_newman.txt', 'w') as f:
    for item in result:
        f.write(', '.join(sorted(list(item))))
        f.write('\n')
    f.close()
!head girvan_newman.txt

Si3aMsOVGSVlsc54iuiPwA, d5WLqmTMvmL7-RmUDVKqqQ
_m1ot2zZetDgjerAD2Sidg, vENR70IrUsDNTDebbuxyQA
23y0Nv9FFWn_3UWudpnFMA, eqWEgMH-DCP74i82BEAZzw
gUu0uaiU7UEUVIgCdnqPVQ, jJDUCuPwVqwjbth3s92whA
F47atsRPw-KHmRVk5exBFw, JeOHA8tW7gr-FDYOcPJoeA
3Vd_ATdvvuVVgn_YCpz8fw, jSbXY_rno4hYHQCFftsWXg
453V8MlGr8y61PpsDAFjKQ, gH0dJQhyKUOVCKQA6sqAnw
46HhzhpBfTdTSB5ceTx_Og, YVQFzWm0H72mLUh-8gzd5w
9W73B44Iw8WslrTNB2CdCg, UmTMCfPlhA6kJLAsLycSfg, Uo5dPwoDpYBzOnmUnjxJ6A
98rLDXbloLXekGjieuQSlA, MJ0Wphhko2-LbJ0uZ5XyQA, QYKexxaOJQlseGWmc6soRg


In [70]:
!tail girvan_newman.txt

98rLDXbloLXekGjieuQSlA, MJ0Wphhko2-LbJ0uZ5XyQA, QYKexxaOJQlseGWmc6soRg
Gr-MqCunME2K_KmsAwjpTA, QRsuZ_LqrRU65dTs5CL4Lw, _6Zg4ukwS0kst9UtkfVw3w, lJFBgSAccsMGwIjfD7LMeQ
CLbpPUqP6XpeAfoqScGaJQ, drTMOo4p8nL0pnMNEyat2A, tX0r-C9BaHYEolRUfufTsQ, xhlcoVm3FOKcxZ0phkdO6Q
Cf0chERnfd06ltnN45xLNQ, CyrRjt_7iJ8_lSHeH1_TlA, JhFK9D3LYl23Se3x4oPUxA, ZW-XoteNlRuuK-19q1spmw, lL-wNa0TKK6LXrlcVmjYrQ
EY8h9IJimXDNbPXVFpYF3A, LiNx18WUre9WFCEQlUhtKA, hilL60vuuh06sMxs6Ckkog, jgoG_hHqnhZvQEoBK0-82w, kwIhn1_cnQeUaLN0CuWWHw, qd16czwFUVHICKF7A4qWsQ
2GUjO7NU88cPXpoffYCU8w, 6YmRpoIuiq8I19Q8dHKTHw, 6xi9tBoZ6r_v41u_XFsSnA, BDmxm7aeWFOLT35gSvkmig, H4EQn0rjFuGRgIm6c9NFLg, H5Asta4LpiKmRhSjWaogIg, XrRLaAeV20MRwdSIGjj2SQ, a48HhwcmjFLApZhiax41IA, angEr2YcXmCl20s8WQu32w, e5sdXDOkCf0sIUAivXVluA, frQs7y5qa-X1pvAM0sJe1w
1st2ltGKJ00ZcRsev-Ieew, 4ZQq0ozRs-gXSz1z55iIDw, HLY9oDcVBH9D25lU4X_V5Q, Hv_q_ZnSIoZwdcoH0CyV2Q, Ih85YhFRDzOnB09yS__94g, KBoIRjxSW7OWczv8OS9Bew, LaiylSIbrA3aPvOYtl-J4A, Z9a1tDT8fVI75qXYwNhPpw, ZEq0WtRJD9Bl_vYgCsbfOg